In [1]:
data_path = "../data/70-15-15"
model_path = "../models"

In [2]:
!pip install tensorflow
#!pip install datasets

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import pandas as pd



In [4]:
treino=pd.read_csv(data_path+'/comunic_treino_aug.csv', encoding='utf-8')
teste=pd.read_csv(data_path+'/comunic_teste.csv', encoding='utf-8')
validacao=pd.read_csv(data_path+'/comunic_valid.csv', encoding='utf-8')

In [5]:
def TFIDF(X_train, X_test,MAX_NB_WORDS=4000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with",str(np.array(X_train).shape[1]),"features")
    return (X_train,X_test)

In [6]:
def make_tfidf(coluna):
    vectorizer = TfidfVectorizer()
    vectors = np.asarray(vectorizer.fit_transform(coluna).todense())
    return vectors

In [7]:
X_train, X_test = TFIDF(treino["text"], teste["text"])
Y_train=treino.label
Y_test=teste.label
X_val=make_tfidf(validacao.text)
Y_val=validacao.label

tf-idf with 4000 features


In [8]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(treino.label), y= treino.label)
class_weights = dict(zip(np.unique(treino.label), class_weights))
class_weights

{0: 1.1276923076923078,
 1: 1.1276923076923078,
 2: 1.1276923076923078,
 3: 1.1276923076923078,
 4: 0.42394447657605555,
 5: 1.1276923076923078,
 6: 1.1276923076923078,
 7: 1.1276923076923078,
 8: 1.1276923076923078,
 9: 1.1276923076923078,
 10: 1.1276923076923078,
 11: 1.1276923076923078,
 12: 1.1276923076923078}

# Definição do modelo de classificacao DNN

In [9]:
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import optimizers

from tensorflow.python.keras.callbacks import EarlyStopping

early_stopping =  EarlyStopping(
    monitor='val_accuracy', 
    patience=4
)

def Build_Model_DNN_Text(shape, nClasses, dropout=0.4):
    model = models.Sequential()
    model.add(layers.Dense(4000,input_dim=shape,activation='relu'))
    model.add(layers.Dense(shape,activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1024,activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dropout(dropout))    
    model.add(layers.Dense(1024,activation='relu'))
    model.add(layers.Dropout(dropout))    
    model.add(layers.Dense(nClasses, activation='sigmoid'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
    model.summary()
    return model

In [10]:
from sklearn import metrics
model_DNN = Build_Model_DNN_Text(X_train.shape[1], 13)
model_DNN.fit(X_train, Y_train,
                              validation_data=(X_test, Y_test),
                              epochs=50,
                              batch_size=128,
                              verbose=2,
                              class_weight=class_weights)
                              #callbacks=[early_stopping])
predicted = model_DNN.predict(X_test)
Y_pred = np.argmax(predicted, axis=1)
print(metrics.classification_report(Y_test, Y_pred))

2023-03-07 00:17:27.947129: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-07 00:17:27.951120: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-07 00:17:27.951246: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-07 00:17:27.951823: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4000)              16004000  
_________________________________________________________________
dense_1 (Dense)              (None, 4000)              16004000  
_________________________________________________________________
dropout (Dropout)            (None, 4000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              4097024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               131200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0

/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Métricas

In [11]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score, hamming_loss
precision = precision_score(Y_test, Y_pred,average='micro')
recall = recall_score(Y_test, Y_pred,average='micro')
f1_micro_average = f1_score(y_true=Y_test, y_pred=Y_pred, average='micro')
#roc_auc = roc_auc_score(Y_test, Y_pred, multi_class ='ovr', average = 'micro')
accuracy = accuracy_score(Y_test, Y_pred)
hloss = hamming_loss(Y_test, Y_pred)

In [12]:
print("Acurácia:",accuracy)
print("Precisão:",precision)
print("Recall:",recall)
print("F1:",f1_micro_average)
#print("Roc_auc:",roc_auc)
print("Hamming Loss:",hloss)

Acurácia: 0.7901234567901234
Precisão: 0.7901234567901234
Recall: 0.7901234567901234
F1: 0.7901234567901234
Hamming Loss: 0.20987654320987653
